In [1]:
import pandas as pd
from datetime import datetime
import requests
import yfinance as yf
import quandl
import numpy as np
from binance.client import Client
import pickle
from datetime import datetime, date, time
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 20)

In [3]:
# All data sources should have date index:
def fill_missing_data(df, freq):
    idx = pd.date_range(df.index[0], df.index[-1], freq = freq)
    try: 
        print(np.sum(idx == df.index) == df.shape[0])
    except:
        df = df.reindex(idx, fill_value = np.nan)
        df = df.interpolate(method = "spline", order = 3, s = 0.)
        print(np.sum(idx == df.index) == df.shape[0], "missing data was corrected")
    return df

# BTC Daily and 5-m Data

In [4]:
# API:
binance_api_key = 'Your Binance Api Key'    #Enter your own API-key here
binance_api_secret = 'Your Binance Api Key' #Enter your own API-secret here
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)

# df_5m:
klines = binance_client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_5MINUTE, "1 Jan, 2017")
df_5m = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
df_5m['timestamp'] = pd.to_datetime(df_5m['timestamp'], unit='ms')

# # df_1d:
klines = binance_client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1DAY, "1 Jan, 2017")
df_1d = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
df_1d['timestamp'] = pd.to_datetime(df_1d['timestamp'], unit='ms')

In [5]:
# Create Daily Returns Series:

## Create the copy of the raw data and operate on the copy:
market_daily = df_1d.copy()

## Create returns dataframe from copy:
market_daily.set_index(market_daily.timestamp, inplace = True)
market_daily.drop(columns=["timestamp"], inplace = True)
market_daily = market_daily.astype(float)
market_daily = fill_missing_data(market_daily, '1d')
btc_ret = pd.DataFrame(100*np.log(market_daily.close/market_daily.close.shift(1)),\
     index = market_daily.index)
btc_ret.dropna(inplace = True)
btc_ret.index = pd.to_datetime(btc_ret.index).date
btc_ret.rename({"close":"returns"}, axis=1, inplace=True)

True


In [6]:
# pickle it for further use in emergency:
with open("./btc_returns", "wb") as file:
    pickle.dump(btc_ret, file)
file.close()
with open("./btc_price_data", "wb") as file:
    pickle.dump(market_daily, file)
file.close()
with open("./btc_price_5m", "wb") as file:
    pickle.dump(df_5m, file)
file.close()

# BTC Realized Volatility: daily, weekly, monthly:

In [7]:
from pandas.core import window


hf_data = df_5m.copy()
hf_data.set_index('timestamp', inplace = True)
hf_data = hf_data.astype(float)

hf_data = fill_missing_data(hf_data, '5min')

hf_ret = pd.DataFrame(100*np.log(hf_data.close.copy()/hf_data.close.copy().shift(1)), \
    index = hf_data.index)
# missing = pd.date_range(start = '2017-12-08', end = '2022-06-17' ).difference(hf_ret.index)
# print(missing)

# Get Daily Realized Volatility:
hf_rv = ((hf_ret**2).rolling(window=288).sum())
hf_rv.dropna(inplace=True)
end_of_the_day = time(23,55,0)
rv_daily = hf_rv.loc[end_of_the_day]
rv_daily = rv_daily.rename({'close':'rv_d'}, axis = 1)

# Check missing and interpolate:
rv_daily = fill_missing_data(rv_daily,"1d")

# Add daily, weekly and monthly to the dataframe:
# Put Indices to The Same Formate And Add RV to df_expl:
rv_daily.set_index(pd.to_datetime(rv_daily.index).date, inplace=True)

# Get weekly realized volatility:
rv_weekly = rv_daily.rolling(window = 7).mean()
rv_monthly = rv_daily.rolling(window = 30).mean()
rv_weekly = fill_missing_data(rv_weekly, "1d")
rv_monthly = fill_missing_data(rv_monthly, "1d")
rv_weekly.rename({"rv_d":"rv_w"}, axis=1, inplace=True)
rv_monthly.rename({"rv_d":"rv_m"}, axis=1, inplace=True)

# Put monthly, weekly daily together:
rv_total = pd.concat([rv_daily, rv_weekly, rv_monthly, btc_ret], axis = 1)
rv_total.dropna(inplace=True)
rv_total.index = pd.to_datetime(rv_total.index)
# pickle
with open("./realized_vols", "wb") as file:
    pickle.dump(rv_total, file)
file.close()

True missing data was corrected
True
True
True


In [8]:
# Checking correctness of aggregation:
(hf_ret**2).rolling(288).sum().loc["2017-09-18"].iloc[-1][0] ==\
    (hf_ret.loc[pd.to_datetime("2017-09-18 00:00:00"):\
        pd.to_datetime("2017-09-18 23:55:00")]**2).sum()[0]

True

In [9]:
#### Note: No missing value in rv_total!

# Economic and Financial variables

In [10]:
from fredapi import Fred
fred = Fred(api_key='insert your api key')
ffer = pd.DataFrame(fred.get_series('dff', observation_start='2000-01-01'),
 columns = ["FFER"])

In [11]:
# Stock, Crypto and Commodity Market Data from YF and NASDAQ:
data = yf.download("CNYUSD=X SPY NDAQ ^DJI CL=F ^VIX ^GVZ XRP-USD BTC=F", start = "2000-01-01")
data = data['Adj Close']
finecon = pd.concat([ffer, data], axis = 1)

[*********************100%***********************]  9 of 9 completed


# Blockchain Info

In [12]:

ids = ['BCHAIN/ATRCT',	'BCHAIN/AVBLS',	'BCHAIN/BLCHS',	'BCHAIN/CPTRA',	'BCHAIN/CPTRV',	'BCHAIN/DIFF',	
                    'BCHAIN/ETRAV',	
    'BCHAIN/ETRVU',	'BCHAIN/HRATE',	'BCHAIN/MIREV',	'BCHAIN/MKPRU',	
                    'BCHAIN/MKTCP',	
    'BCHAIN/MWNTD',	'BCHAIN/MWNUS',	'BCHAIN/MWTRV',	'BCHAIN/NADDU',
                	'BCHAIN/NTRAN',	
    'BCHAIN/NTRAT',	'BCHAIN/NTRBL',	'BCHAIN/NTREP',	'BCHAIN/TOTBC',	'BCHAIN/TOUTV',	
                    'BCHAIN/TRFEE',	
    'BCHAIN/TRFUS',	'BCHAIN/TRVOU']


In [13]:
blockchain_info = quandl.get(ids, authtoken="insert your authotoken")

In [14]:
blockchain_info.columns = [ 'ATRCT',	'AVBLS','BLCHS',
	'CPTRA',	'CPTRV',	'DIFF',	'ETRAV',	'ETRVU',	'HRATE',	'MIREV',	'MKPRU',	'MKTCP',
    	'MWNTD',	'MWNUS',	'MWTRV',	'NADDU',	'NTRAN',	'NTRAT',	'NTRBL',	'NTREP',	
        'TOTBC',	'TOUTV',	'TRFEE',	'TRFUS',	'TRVOU']


In [15]:
blockchain_info.isna().sum()

ATRCT    391
AVBLS      2
BLCHS      2
CPTRA      2
CPTRV      5
        ... 
TOTBC      1
TOUTV      2
TRFEE      2
TRFUS      2
TRVOU      1
Length: 25, dtype: int64

# Risks and Uncertainties

### EPU

In [16]:
# Chine Economic Policy Uncertainty Data:
dls = "https://economicpolicyuncertaintyinchina.weebly.com/uploads/1/2/2/7/122762465/cnepu_daily_18_june_2022_updated.xlsx"
resp = requests.get(dls)

output = open('./fin.xlsx', 'wb')
output.write(resp.content)
output.close()

epu = pd.read_excel("./fin.xlsx", parse_dates = ["Date"])
epu.dropna(inplace=True)
epu.Date = pd.to_datetime(epu.Date)
epu.set_index(epu.Date, inplace = True)
epu.drop(columns = ["Date"], inplace = True)
np.sum(epu.isna())
epu.columns = ["CNEPU"]


### GPRD

In [17]:
# Geopolitical Risk Data:
url = "https://www.matteoiacoviello.com/gpr_files/data_gpr_daily_recent.xls"
resp = requests.get(url)
output = open('gpr.xls', 'wb')
output.write(resp.content)
output.close()

gpr = pd.read_excel("./gpr.xls", parse_dates=["date"])
gpr.date = pd.to_datetime(gpr.date)
gpr.set_index(gpr.date, inplace = True)
gpr.drop(columns = ["date"], inplace = True)
gprd = pd.DataFrame(gpr.GPRD, index=gpr.index)
np.sum(gprd.isna())

GPRD    0
dtype: int64

In [18]:
risks = pd.concat([gprd, epu], axis=1)

In [19]:
pd.set_option('display.max_rows', 20)

# Public Opinion

### Gtrends

In [20]:
gtrends = pd.read_excel("./gtrends.xlsx", parse_dates=["Date"])
gtrends = gtrends.set_index(gtrends.Date)
gtrends.drop(columns=["Date"], inplace=True)
nulls = [type(gtrends["gtrends"][i])==str for i in range(len(gtrends))]
gtrends.iloc[nulls, 0] = np.nan
gtrends["gtrends"] = gtrends.gtrends.astype(float)

### Tweets

In [21]:
tweets = pd.read_excel("./tweets.xlsx", sheet_name="tweets")
tweets = tweets.set_index(tweets.Date)
tweets.drop(columns=["Date"], inplace=True)
nulls = [type(tweets["tweet"][i])!=int for i in range(len(tweets))]
tweets.iloc[nulls, 0] = np.nan
tweets["tweet"] = tweets.tweet.astype(float)
tweets = tweets.interpolate(method = "spline", order = 3, s = 0.)
np.sum(tweets.isna())

tweet    0
dtype: int64

In [22]:
public_opinion = pd.concat([gtrends, tweets], axis=1)

# Merge Data in one dataframe

In [23]:
sources = [blockchain_info, finecon, risks, public_opinion, rv_total]
data_full = pd.concat(sources, axis = 1)

In [24]:
start = [data_full.loc[data_full[i].notna().to_numpy()].index[0] for i in data_full.columns]
end = [data_full.loc[data_full[i].notna().to_numpy()].index[-1] for i in data_full.columns]
idx = pd.date_range(start = max(start), end = min(end), freq='D')
data_full = data_full.loc[idx]
data_full = data_full.fillna(method='ffill')
# data_full.isna().sum()

In [25]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

In [26]:
# data_full.isna().sum().tail(20)

In [27]:
# Fill missing data by interpolation:
sources = [blockchain_info, finecon, risks, rv_total, public_opinion]
blockchain_info, finecon, risks, rv_total, public_opinion = \
    [fill_missing_data(i, "1d") for i in sources]
data = pd.concat(sources, axis = 1)

True
True
True
True
True


In [28]:
# Double Check the Data:
np.sum(data_full["CNEPU"] == epu.CNEPU.loc[data_full.index]) \
    == data_full.shape[0]
np.sum(btc_ret.loc[data_full.index]==data_full[["returns"]])==len(data_full)
 

returns    True
dtype: bool

In [29]:
data_full.to_excel("data_full.xlsx")

In [30]:
data_full.columns

Index(['ATRCT', 'AVBLS', 'BLCHS', 'CPTRA', 'CPTRV', 'DIFF', 'ETRAV', 'ETRVU',
       'HRATE', 'MIREV', 'MKPRU', 'MKTCP', 'MWNTD', 'MWNUS', 'MWTRV', 'NADDU',
       'NTRAN', 'NTRAT', 'NTRBL', 'NTREP', 'TOTBC', 'TOUTV', 'TRFEE', 'TRFUS',
       'TRVOU', 'FFER', 'BTC=F', 'CL=F', 'CNYUSD=X', 'NDAQ', 'SPY', 'XRP-USD',
       '^DJI', '^GVZ', '^VIX', 'GPRD', 'CNEPU', 'gtrends', 'tweet', 'rv_d',
       'rv_w', 'rv_m', 'returns'],
      dtype='object')